In [70]:
# Huffman encoding implementation

import heapq, time
from collections import Counter

class Node:
    def __init__(self, freq, char=None, left=None, right=None):
        self.freq = freq
        self.char = char
        self.left = left
        self.right = right
    
    # Define comparison operators for heapq
    def __lt__(self, other):
        return self.freq < other.freq


def build_huffman_tree(text):
    # create dict of characters and frequencies
    frequency = Counter(text)
    # create a heap from the values in frequency
    heap = [Node(freq, char) for char, freq in frequency.items()]
    heapq.heapify(heap)
        
    
    while len(heap) > 1:
        # take the 2 lowest frequencies
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        # make new node for the combined frequencies
        combined = Node(left.freq + right.freq, left=left, right=right)
        # push the combined value into the heap and continue
        heapq.heappush(heap, combined)
    
    
    return heap[0] # root of the tree


def build_codes(node, prefix="", codebook=None):
    if codebook is None:
        codebook = {}
    if node.char is not None:
        codebook[node.char] = prefix
        # add char and prefix val into dict
    else:
        # traverse tree until a char is found
        build_codes(node.left, prefix + "0", codebook)
        build_codes(node.right, prefix + "1", codebook)
    
    # could really just print this dict out instead of a whole tree
    # you know what I will
    return codebook

def huffman_encoding(text):
    if not text:
        return "", None , {}
    
    root = build_huffman_tree(text)
    codebook = build_codes(root)
    
    # edge casing :)
    if len(set(text)) == 1:
        return "0" * len(text) , root, {text[0]: len(text)}
    
    # take dict and go through it add appropriate encoding for each character
    encoded_text = ''.join(codebook[char] for char in text)
    return encoded_text, root, codebook

def huffman_decoding(encoded_text, root):
    # stupid edge case, why use huffman if only one char
    if len(set(encoded_text)) == 1:
        node = root
        return (node.char) * len(encoded_text)
    if not encoded_text or not root:
        return ""
    decoded_text = []
    node = root
    # go through encoded text bit by bit and add character when found
    # repeat for all bits
    for bit in encoded_text:
        node = node.left if bit == "0" else node.right
        if node.char is not None:
            decoded_text.append(node.char)
            node = root
    return ''.join(decoded_text)

def huffman_encoding_with_tree(text):
    encoded_text, root, codebook = huffman_encoding(text)
    tree_str = print_huffman_tree(root)
    return encoded_text, tree_str, root, codebook

# not pretty but really? print the tree?
def print_huffman_tree(node, indent=""):
    if node is not None:
        tree_str = f"{indent}{node.freq} {'('+node.char+')' if node.char else ''}\n"
        if node.left or node.right:
            tree_str += f"{indent}├── Left: 0\n"
            tree_str += print_huffman_tree(node.left, indent + "│   ")
            tree_str += f"{indent}└── Right: 1\n"
            tree_str += print_huffman_tree(node.right, indent + "    ")
        return tree_str
    return ""


text = "tested"
start = time.time()
encoded_text, tree_str, root , dict = huffman_encoding_with_tree(text)
end = time.time()
print ( "\nTime taken", (end-start) * 1000, " ms")
print (dict)
print("Encoded Text:", encoded_text)
print("Huffman Tree:\n", tree_str)

start = time.time()
decoded_text = huffman_decoding(encoded_text, root)
end = time.time()
print ( "\nTime taken", (end-start) * 1000, " ms")
print(f"Decoded:{decoded_text}")



Time taken 0.0  ms
{'s': '00', 'd': '01', 'e': '10', 't': '11'}
Encoded Text: 111000111001
Huffman Tree:
 6 
├── Left: 0
│   2 
│   ├── Left: 0
│   │   1 (s)
│   └── Right: 1
│       1 (d)
└── Right: 1
    4 
    ├── Left: 0
    │   2 (e)
    └── Right: 1
        2 (t)


Time taken 0.0  ms
Decoded:tested


In [71]:
import unittest

class TestHuffmanCoding(unittest.TestCase):

    def test(self):
        text = "testing"
        encoded_text, tree, code = huffman_encoding(text)
        decoded_text = huffman_decoding(encoded_text, tree)
        self.assertEqual(text, decoded_text)

    def test_empty(self):
        text = ""
        encoded_text, tree, code = huffman_encoding(text)
        decoded_text = huffman_decoding(encoded_text, tree)
        self.assertEqual(text, decoded_text)
    
    def test_numbers_with_special(self):
        text = "1 2 3 4 :)"
        encoded_text, tree, code = huffman_encoding(text)
        decoded_text = huffman_decoding(encoded_text, tree)
        self.assertEqual(text, decoded_text)

    def test_repeated_char(self):
        text = "aaaaaaa"
        encoded_text, tree, code = huffman_encoding(text)
        decoded_text = huffman_decoding(encoded_text, tree)
        self.assertEqual(text, decoded_text)
        
    def test_pattern(self):
        text = "abc123abc123abc"
        encoded_text, tree, code = huffman_encoding(text)
        decoded_text = huffman_decoding(encoded_text, tree)
        self.assertEqual(text, decoded_text)
        


In [72]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.002s

OK
